In [1]:
%pip install requests

Defaulting to user installation because normal site-packages is not writeable
You should consider upgrading via the '/Library/Developer/CommandLineTools/usr/bin/python3 -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


In [2]:
import requests
import json
import os

In [3]:
#Define path
SECRETS_FILE = '/Users/gmccurdy/repos/curriculum/scripts/utilities/secrets.json'

# Load secrets
with open(SECRETS_FILE, 'r') as file:
    secrets = json.load(file)

# Assign variables from the secrets
API_URL = secrets.get('API_URL')
API_KEY = secrets.get('API_KEY')
COURSE_ID = secrets.get('COURSE_ID_PRECALCULUS')

#Define headers
headers = {
    'Authorization': f'Bearer {API_KEY}',
    'Content-Type': 'application/json'
}

In [4]:
# Create the quiz
quiz_title = input('Quiz Title')
quiz_description = input('Quiz Description')
quiz_data = {
    'quiz': {
        'title': quiz_title,
        'description': f'<p>{quiz_description}</p>',
        'quiz_type': 'assignment',
        'published': False
    }
}

create_quiz_url = f'{API_URL}/api/v1/courses/{COURSE_ID}/quizzes'
response = requests.post(create_quiz_url, headers=headers, json=quiz_data)

if response.status_code == 200:
    quiz = response.json()
    quiz_id = quiz['id']
    print(f'Quiz created successfully with ID: {quiz_id}')
else:
    print('Failed to create quiz')
    print(f"Status Code: {response.status_code}")
    print(response.text)

Quiz created successfully with ID: 44083


In [10]:
multiple_choice_questions = [
    {
        'question_name': 'Sine of Negative Angle -π/3',
        'question_type': 'multiple_choice_question',
        'question_text': r'''What is \( \sin\left(-\frac{\pi}{3}\right) \)?''',
        'points_possible': 1,
        'answers': [
            {'text': r'\( \frac{\sqrt{3}}{2} \)', 'weight': 0},
            {'text': r'\( -\frac{\sqrt{3}}{2} \)', 'weight': 100},  # Correct answer
            {'text': r'\( \frac{1}{2} \)', 'weight': 0},
            {'text': r'\( -\frac{1}{2} \)', 'weight': 0},
        ]
    },
    {
        'question_name': 'Cotangent of 7π/4',
        'question_type': 'multiple_choice_question',
        'question_text': r'''Evaluate \( \cot\left(\frac{7\pi}{4}\right) \).''',
        'points_possible': 1,
        'answers': [
            {'text': r'\( 1 \)', 'weight': 100},        # Correct answer
            {'text': r'\( -1 \)', 'weight': 0},
            {'text': r'\( \sqrt{3} \)', 'weight': 0},
            {'text': r'\( -\sqrt{3} \)', 'weight': 0},
        ]
    },
    {
        'question_name': 'Secant of -2π/3',
        'question_type': 'multiple_choice_question',
        'question_text': r'''Find \( \sec\left(-\frac{2\pi}{3}\right) \).''',
        'points_possible': 1,
        'answers': [
            {'text': r'\( 2 \)', 'weight': 0},
            {'text': r'\( -2 \)', 'weight': 100},       # Correct answer
            {'text': r'\( \frac{2}{\sqrt{3}} \)', 'weight': 0},
            {'text': r'\( -\frac{2}{\sqrt{3}} \)', 'weight': 0},
        ]
    },
    {
        'question_name': 'Cosine of 13π/6',
        'question_type': 'multiple_choice_question',
        'question_text': r'''Evaluate \( \cos\left(\frac{13\pi}{6}\right) \).''',
        'points_possible': 1,
        'answers': [
            {'text': r'\( \frac{\sqrt{3}}{2} \)', 'weight': 100},  # Correct answer
            {'text': r'\( -\frac{\sqrt{3}}{2} \)', 'weight': 0},
            {'text': r'\( \frac{1}{2} \)', 'weight': 0},
            {'text': r'\( -\frac{1}{2} \)', 'weight': 0},
        ]
    },
    {
        'question_name': 'Sine of 5π/2',
        'question_type': 'multiple_choice_question',
        'question_text': r'''What is \( \sin\left(\frac{5\pi}{2}\right) \)?''',
        'points_possible': 1,
        'answers': [
            {'text': r'\( -1 \)', 'weight': 0},
            {'text': r'\( 1 \)', 'weight': 100},        # Correct answer
            {'text': r'\( 0 \)', 'weight': 0},
            {'text': r'\( \frac{1}{2} \)', 'weight': 0},
        ]
    },
    {
        'question_name': 'Cosecant of -π/2',
        'question_type': 'multiple_choice_question',
        'question_text': r'''Find \( \csc\left(-\frac{\pi}{2}\right) \).''',
        'points_possible': 1,
        'answers': [
            {'text': r'\( 1 \)', 'weight': 0},
            {'text': r'\( -1 \)', 'weight': 100},       # Correct answer
            {'text': r'\( \text{Undefined} \)', 'weight': 0},
            {'text': r'\( 0 \)', 'weight': 0},
        ]
    },
]

In [6]:
essay_questions = [
    
]

In [7]:
numerical_questions = [

]

In [11]:
# Combine all questions, including the essay question
questions = multiple_choice_questions + numerical_questions + essay_questions

# Add questions to the quiz
for question in questions:
    create_question_url = f'{API_URL}/api/v1/courses/{COURSE_ID}/quizzes/{quiz_id}/questions'

    # Base question data
    question_data = {
        'question': {
            'question_name': question['question_name'],
            'question_type': question['question_type'],
            'question_text': question['question_text'],
            'points_possible': question.get('points_possible', 1)
            # Do not include 'answers' here
        }
    }

    # Include 'answers' only if they exist and the question is not an essay question
    if 'answers' in question and question['question_type'] != 'essay_question':
        question_data['question']['answers'] = question['answers']

    # Include 'general_feedback' if it exists (for essay questions)
    if 'general_feedback' in question:
        question_data['question']['general_feedback'] = question['general_feedback']

    # Send the request to create the question
    response = requests.post(create_question_url, headers=headers, json=question_data)

    if response.status_code == 201 or response.status_code == 200:
        print(f"Added question '{question['question_name']}' successfully.")
    else:
        print(f"Failed to add question '{question['question_name']}'")
        print(f"Status Code: {response.status_code}")
        print(response.text)

Added question 'Sine of Negative Angle -π/3' successfully.
Added question 'Cotangent of 7π/4' successfully.
Added question 'Secant of -2π/3' successfully.
Added question 'Cosine of 13π/6' successfully.
Added question 'Sine of 5π/2' successfully.
Added question 'Cosecant of -π/2' successfully.
